In [1]:
import numpy as np # библиотека для работы с чиселками
import pandas as pd # data processing, работа с CSV файлами
import matplotlib.pyplot as plt # для графики
import seaborn as sns # аналогично

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

Откроем датасет и посмотрим первые 5 его строчек

In [2]:
dataset = pd.read_csv('laptop_price.csv', encoding='ISO-8859-1')
dataset.head(5)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [3]:
dataset.describe()

,laptop_ID,Inches,Price_euros
count,1303.000000,1303.000000,1303.000000
mean,660.155794,15.017191,1123.686992
std,381.172104,1.426304,699.009043
min,1.000000,10.100000,174.000000
25%,331.500000,14.000000,599.000000
50%,659.000000,15.600000,977.000000
75%,990.500000,15.600000,1487.880000
max,1320.000000,18.400000,6099.000000


Т.к. необходимо решить задачу классификации, то разделим цены на 3 категории: низкие, средние и высокие

Для задачи регрессии будем предсказывать цену ноутбука по остальным характеристикам

In [4]:
dataset['Price_category'] = pd.qcut(dataset['Price_euros'], q=3, labels=['Low', 'Medium', 'High'])


Подготовим данные

In [5]:
# Удаление ненужных столбцов
dataset = dataset.drop(columns=['laptop_ID'])
new_dataset = dataset.copy()

# Кодирование категориальных признаков
categorical_columns = ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight']
label_encoders = {}
for column in categorical_columns:
    le = LabelEncoder()
    new_dataset[column] = le.fit_transform(new_dataset[column])
    label_encoders[column] = le  # Сохраняем энкодеры, если понадобится обратное преобразование


# Разделение на признаки и целевую переменную
X = new_dataset.drop(columns=['Price_euros', 'Price_category'])
y_classification = new_dataset['Price_category']
y_regression = new_dataset['Price_euros']

# Масштабирование числовых признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Разделим датасет на обучающую и тестовую выборку

In [6]:
X_train, X_test, y_train_classification, y_test_classification = train_test_split(X_scaled, y_classification, test_size=0.2, random_state=42)

In [7]:
X_train.shape

(1042, 11)

In [8]:
y_train_classification


10         Low
147        Low
1287    Medium
767     Medium
816       High
         ...  
1095    Medium
1130       Low
1294       Low
860     Medium
1126       Low
Name: Price_category, Length: 1042, dtype: category
Categories (3, object): ['Low' < 'Medium' < 'High']

##### Линейная регрессия в чистом виде не предназначена для классификации. Это подход только для регрессии. Логистическая регрессия не предназначена для задач регрессии. Поэтому для задачи классификации будет рассматриваться метод логистической регрессии, а для задачи регрессии - алгоритм линейной регрессии. 

Обучим модель Logistic regression

In [9]:
model_classification = LogisticRegression(max_iter=1000)

model_classification.fit(X_train, y_train_classification)

LogisticRegression(max_iter=1000)

Оценим работу классификатора

In [10]:
train_predict_classification = model_classification.predict(X_train)
test_predict_classification = model_classification.predict(X_test)

In [11]:
# Расчет метрик
train_f1_classification = f1_score(y_train_classification, train_predict_classification, average='micro')
train_accuracy_classification = accuracy_score(y_train_classification, train_predict_classification)
train_precision_classification = precision_score(y_train_classification, train_predict_classification, average='micro')
train_recall_classification = recall_score(y_train_classification, train_predict_classification, average='micro')

test_f1_classification = f1_score(y_test_classification, test_predict_classification, average='micro')
test_accuracy_classification = accuracy_score(y_test_classification, test_predict_classification)
test_precision_classification = precision_score(y_test_classification, test_predict_classification, average='micro')
test_recall_classification = recall_score(y_test_classification, test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', train_f1_classification, train_accuracy_classification, train_precision_classification, train_recall_classification)
print('Test', test_f1_classification, test_accuracy_classification, test_precision_classification, test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.7226487523992322 0.7226487523992322 0.7226487523992322 0.7226487523992322
Test 0.6896551724137931 0.6896551724137931 0.6896551724137931 0.6896551724137931


### Теперь посмотрим работу алгоритма Linear regression для задачи регрессии

Разделим датасет на обучающую и тестовую выборку

In [12]:
X_train, X_test, y_train_regression, y_test_regression = train_test_split(X_scaled, y_regression, test_size=0.2, random_state=42)

Обучим модель Linear regression

In [13]:
model_regression = LinearRegression()

model_regression.fit(X_train, y_train_regression)

LinearRegression()

Оценим работу

In [14]:
train_predict_regression = model_regression.predict(X_train)
test_predict_regression = model_regression.predict(X_test)

In [15]:
# Расчет метрик
train_mse_regression = mean_squared_error(y_train_regression, train_predict_regression)
train_mae_regression = mean_absolute_error(y_train_regression, train_predict_regression)

test_mse_regression = mean_squared_error(y_test_regression, test_predict_regression)
test_mae_regression = mean_absolute_error(y_test_regression, test_predict_regression)



print('Метрика:          MSE            MAE')
print('Train', train_mse_regression, train_mae_regression)
print('Test', test_mse_regression, test_mae_regression)

Метрика:          MSE            MAE
Train 244676.72122382946 369.53648644328274
Test 306598.7169341574 377.073990733218


### Вывод:
1. Качество классификации (F1, Accuracy, Precision, Recall):
На тренировочной выборке метрики равны 0.72. Это говорит о среднем качестве модели на тренировочных данных. Она не слишком хорошо различает классы, возможно, из-за недостаточной сложности модели или наличия значительного уровня шума в данных.
На тестовой выборке метрики снижаются до 0.69, что указывает на небольшую потерю качества при переходе к неизвестным данным. Это приемлемо, но может сигнализировать о необходимости улучшения модели или её гиперпараметров.

2. Качество регрессии (MSE, MAE):
На тренировочной выборке значения MSE (244 677) и MAE (369.54) достаточно высоки, что свидетельствует о значительных ошибках в предсказаниях модели.
На тестовой выборке значения MSE (306,599) и MAE (377.07) увеличиваются, что указывает на ухудшение качества модели на новых данных.

Качество модели недостаточно высокое как в задаче классификации, так и регрессии, наблюдается тенденция к снижению точности на тестовых данных, что говорит о необходимости увеличения сложности модели и тюнинга гиперпараметров. Для задач регрессии текущая модель (линейная регрессия) может быть недостаточно сложной для отражения зависимостей в данных.
Результаты текущих моделей удовлетворительны как базовый уровень, но их можно значительно улучшить за счёт применения более сложных методов, тщательной настройки гиперпараметров и оптимальной обработки данных.

# Улучшение бейзлайна

Для улучшения бейзлайна модели в задачах классификации и регрессии предлагаю следующие решения:

Удалить столбцы с высокой кореляцией, закодировать категориальные признаки более информативно (используя One-Hot Encoding), а также использовать метод GridSearchCV для поиска лучших параметров.

Для начала посмотрим матрицу кореляций для данного датасета

In [16]:
X.corr()

,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight
Company,1.000000,0.067234,-0.007718,-0.085784,0.094772,0.044593,-0.008399,0.048086,0.026537,0.134290,-0.159906
Product,0.067234,1.000000,0.065594,-0.218974,0.176393,0.143855,0.018305,0.088667,0.094795,0.120562,-0.204379
TypeName,-0.007718,0.065594,1.000000,-0.077428,-0.175618,-0.128174,0.213988,0.019091,-0.204774,0.085223,-0.211832
Inches,-0.085784,-0.218974,-0.077428,1.000000,-0.247841,0.153041,-0.149706,-0.193382,0.215710,0.034528,0.866304
ScreenResolution,0.094772,0.176393,-0.175618,-0.247841,1.000000,0.232751,0.019858,0.059042,0.160816,0.070360,-0.218135
Cpu,0.044593,0.143855,-0.128174,0.153041,0.232751,1.000000,-0.077176,-0.013264,0.490761,0.125374,0.139861
Ram,-0.008399,0.018305,0.213988,-0.149706,0.019858,-0.077176,1.000000,-0.248580,-0.138212,0.008627,-0.174577
Memory,0.048086,0.088667,0.019091,-0.193382,0.059042,-0.013264,-0.248580,1.000000,-0.020784,0.035574,-0.218299
Gpu,0.026537,0.094795,-0.204774,0.215710,0.160816,0.490761,-0.138212,-0.020784,1.000000,0.103853,0.274961
OpSys,0.134290,0.120562,0.085223,0.034528,0.070360,0.125374,0.008627,0.035574,0.103853,1.000000,-0.031506


Можно заметить, что у Weight и Inches коэффициент кореляции 0.86, что достаточно много, значит уберем один из этих столбцов (пусть это будет Weight)

In [17]:
dataset = dataset.drop(columns=['Weight'])

In [18]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [19]:
# Создание пайплайна для классификации
categorical_features = ['Company', 'Product', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys']
numerical_features = ['Inches']

X = dataset.drop(columns=['Price_euros', 'Price_category'])

new_X_train, new_X_test, y_train_regression, y_test_regression = train_test_split(X, y_regression, test_size=0.2, random_state=42)
_, _, y_train_classification, y_test_classification = train_test_split(X, y_classification, test_size=0.2, random_state=42)

# OneHotEncoding для категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Пайплайн для классификации
classification_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Пайплайн для регрессии
regression_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Параметры для GridSearchCV
param_grid_classification = {
    'classifier__C': [0.1, 1, 10],
    'classifier__solver': ['liblinear', 'lbfgs'],
    'classifier__penalty': ['l2']
}

# LinearRegression в sklearn не имеет гиперпараметров для настройки через GridSearchCV

# Кросс-валидация для классификации
grid_search_classification = GridSearchCV(classification_pipeline, param_grid_classification)
grid_search_classification.fit(new_X_train, y_train_classification)

# Кросс-валидация для регрессии
regression_pipeline.fit(new_X_train, y_train_regression)

# Лучшие параметры
print("Best parameters for classification:", grid_search_classification.best_params_)


Best parameters for classification: {'classifier__C': 10, 'classifier__penalty': 'l2', 'classifier__solver': 'liblinear'}


In [20]:
preprocessor

ColumnTransformer(transformers=[('num', StandardScaler(), ['Inches']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['Company', 'Product', 'TypeName',
                                  'ScreenResolution', 'Cpu', 'Ram', 'Memory',
                                  'Gpu', 'OpSys'])])

In [21]:
preprocessor.fit(new_X_train)

X_train_preprocessed = preprocessor.transform(new_X_train)
X_test_preprocessed = preprocessor.transform(new_X_test)

Обучим модели и оценим их работу

In [22]:
new_model_classification = LogisticRegression(C=10, penalty='l2', solver='liblinear')

new_model_classification.fit(X_train_preprocessed, y_train_classification)

LogisticRegression(C=10, solver='liblinear')

In [23]:
new_model_regression = LinearRegression()

new_model_regression.fit(X_train_preprocessed, y_train_regression)

LinearRegression()

In [24]:
new_train_predict_classification = new_model_classification.predict(X_train_preprocessed)
new_test_predict_classification = new_model_classification.predict(X_test_preprocessed)

In [25]:
# Расчет метрик
new_train_f1_classification = f1_score(y_train_classification, new_train_predict_classification, average='micro')
new_train_accuracy_classification = accuracy_score(y_train_classification, new_train_predict_classification)
new_train_precision_classification = precision_score(y_train_classification, new_train_predict_classification, average='micro')
new_train_recall_classification = recall_score(y_train_classification, new_train_predict_classification, average='micro')

new_test_f1_classification = f1_score(y_test_classification, new_test_predict_classification, average='micro')
new_test_accuracy_classification = accuracy_score(y_test_classification, new_test_predict_classification)
new_test_precision_classification = precision_score(y_test_classification, new_test_predict_classification, average='micro')
new_test_recall_classification = recall_score(y_test_classification, new_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', new_train_f1_classification, new_train_accuracy_classification, new_train_precision_classification, new_train_recall_classification)
print('Test', new_test_f1_classification, new_test_accuracy_classification, new_test_precision_classification, new_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.982725527831094 0.982725527831094 0.982725527831094 0.982725527831094
Test 0.7969348659003831 0.7969348659003831 0.7969348659003831 0.7969348659003831


In [26]:
new_train_predict_regression = new_model_regression.predict(X_train_preprocessed)
new_test_predict_regression = new_model_regression.predict(X_test_preprocessed)

In [27]:
# Расчет метрик
new_train_mse_regression = mean_squared_error(y_train_regression, new_train_predict_regression)
new_train_mae_regression = mean_absolute_error(y_train_regression, new_train_predict_regression)

new_test_mse_regression = mean_squared_error(y_test_regression, new_test_predict_regression)
new_test_mae_regression = mean_absolute_error(y_test_regression, new_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', new_train_mse_regression, new_train_mae_regression)
print('Test', new_test_mse_regression, new_test_mae_regression)

Метрика:          MSE            MAE
Train 7394.854727185142 42.12358168109574
Test 81385.29275362074 189.7951674472844


### Вывод:

Классификация: точность на тренировочной выборке достигла очень высокого значения (98.27%). Это говорит о том, что модель успешно обучилась распознавать паттерны в данных. Точность на тестовой выборке составила около 79.69%. Несмотря на снижение показателей по сравнению с тренировочной выборкой, модель демонстрирует хорошую обобщающую способность.
Регрессия: MSE и MAE на тренировочной выборке имеют значения 7394.85 и 42.12 соответственно, что указывают на то, что модель почти идеально предсказывает целевую переменную на тренировочной выборке, что может быть признаком переобучения. MSE и MAE на тестовой выборке: 81385.29 и 189.80 - увеличились, что ожидаемо из-за отсутствия информации о данных из тестовой выборки при обучении. Тем не менее, это указывает на допустимый уровень ошибки.

Метрики классификации и регрессии на тестовой выборке демонстрируют небольшое улучшение, но все равно точность не очень выслкая (примерно 0.8). Снижение метрик классификации на тестовой выборке (по сравнению с тренировочной) может свидетельствовать о возможности переобучения.

### Имплементация алгоритма

In [28]:
from collections import Counter

In [29]:
class LOGISTICREGRESSION:
    def __init__(self, lr=0.01, epochs=1000, regularization=None, reg_strength=0.0):
        self.lr = lr
        self.epochs = epochs
        self.weights = None
        self.bias = None
        self.regularization = regularization
        self.reg_strength = reg_strength
        self.momentum = 0.9
        self.beta = 0.999
        self.eps = 1e-8
        self.m = None  # Для накопления градиента (m)
        self.v = None  # Для накопления квадрата градиента (v)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.random.randn(n_features) * np.sqrt(1 / n_features)
        self.bias = 0
        self.m = np.zeros_like(self.weights)
        self.v = np.zeros_like(self.weights)

        for epoch in range(self.epochs):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(linear_model)

            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Адаптивные шаги обучения (Adam)
            self.m = self.momentum * self.m + (1 - self.momentum) * dw
            self.v = self.beta * self.v + (1 - self.beta) * (dw ** 2)

            m_corr = self.m / (1 - self.momentum ** (epoch + 1))
            v_corr = self.v / (1 - self.beta ** (epoch + 1))

            dw_adam = m_corr / (np.sqrt(v_corr) + self.eps)

            self.weights -= self.lr * dw_adam
            self.bias -= self.lr * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self.sigmoid(linear_model)
        return np.round(y_predicted)


class LINEARREGRESSION:
    def __init__(self, lr=0.01, epochs=1000, regularization=None, reg_strength=0.0):
        self.lr = lr
        self.epochs = epochs
        self.weights = None
        self.bias = None
        self.regularization = regularization
        self.reg_strength = reg_strength

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y = np.array(y)  # Преобразование в numpy массив
        self.weights = np.random.randn(n_features) * np.sqrt(1 / n_features)
        self.bias = 0

        for epoch in range(self.epochs):
            # Перемешивание данных
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            # Предсказание
            y_predicted = np.dot(X, self.weights) + self.bias

            # Градиенты
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Регуляризация
            if self.regularization == "l2":
                dw += (self.reg_strength / n_samples) * self.weights

            # Обновление параметров
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


In [30]:
from sklearn.preprocessing import LabelEncoder

# Кодирование целевой переменной
label_encoder = LabelEncoder()
y_train_classification_encoded = label_encoder.fit_transform(y_train_classification)
y_test_classification_encoded = label_encoder.fit_transform(y_test_classification)

In [31]:
# Применение собственной реализации для классификации
logreg_classifier = LOGISTICREGRESSION(lr=1e-3, epochs=1000, regularization="l2", reg_strength=0.01)
logreg_classifier.fit(X_train_preprocessed.toarray(), y_train_classification_encoded)

my_train_predict_classification = logreg_classifier.predict(X_train_preprocessed.toarray())
my_test_predict_classification = logreg_classifier.predict(X_test_preprocessed.toarray())

In [32]:
# Расчет метрик
my_train_f1_classification = f1_score(y_train_classification_encoded, my_train_predict_classification, average='micro')
my_train_accuracy_classification = accuracy_score(y_train_classification_encoded, my_train_predict_classification)
my_train_precision_classification = precision_score(y_train_classification_encoded, my_train_predict_classification, average='micro')
my_train_recall_classification = recall_score(y_train_classification_encoded, my_train_predict_classification, average='micro')

my_test_f1_classification = f1_score(y_test_classification_encoded, my_test_predict_classification, average='micro')
my_test_accuracy_classification = accuracy_score(y_test_classification_encoded, my_test_predict_classification)
my_test_precision_classification = precision_score(y_test_classification_encoded, my_test_predict_classification, average='micro')
my_test_recall_classification = recall_score(y_test_classification_encoded, my_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', my_train_f1_classification, my_train_accuracy_classification, my_train_precision_classification, my_train_recall_classification)
print('Test', my_test_f1_classification, my_test_accuracy_classification, my_test_precision_classification, my_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.5 0.5 0.5 0.5
Test 0.38697318007662834 0.38697318007662834 0.38697318007662834 0.38697318007662834


In [33]:

# Применение собственной реализации для регрессии
linreg_regressor = LINEARREGRESSION(lr=1e-2, epochs=1000, regularization="l2", reg_strength=0.1)
linreg_regressor.fit(X_train, y_train_regression)

my_train_predict_regression = linreg_regressor.predict(X_train)
my_test_predict_regression = linreg_regressor.predict(X_test)

In [34]:
# Расчет метрик
my_train_mse_regression = mean_squared_error(y_train_regression, my_train_predict_regression)
my_train_mae_regression = mean_absolute_error(y_train_regression, my_train_predict_regression)

my_test_mse_regression = mean_squared_error(y_test_regression, my_test_predict_regression)
my_test_mae_regression = mean_absolute_error(y_test_regression, my_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', my_train_mse_regression, my_train_mae_regression)
print('Test', my_test_mse_regression, my_test_mae_regression)

Метрика:          MSE            MAE
Train 244901.56493539913 369.61349958628915
Test 305991.1030901578 376.53337010116104


Посмотрим работу алгоритма на улучшенном датасете

In [35]:
# Применение собственной реализации для классификации
logreg_classifier = LOGISTICREGRESSION(lr=1e-3, epochs=1000, regularization="l2", reg_strength=0.01)
logreg_classifier.fit(X_train, y_train_classification_encoded)

new_my_train_predict_classification = logreg_classifier.predict(X_train)
new_my_test_predict_classification = logreg_classifier.predict(X_test)

In [36]:
# Расчет метрик
new_my_train_f1_classification = f1_score(y_train_classification_encoded, new_my_train_predict_classification, average='micro')
new_my_train_accuracy_classification = accuracy_score(y_train_classification_encoded, new_my_train_predict_classification)
new_my_train_precision_classification = precision_score(y_train_classification_encoded, new_my_train_predict_classification, average='micro')
new_my_train_recall_classification = recall_score(y_train_classification_encoded, new_my_train_predict_classification, average='micro')

new_my_test_f1_classification = f1_score(y_test_classification_encoded, new_my_test_predict_classification, average='micro')
new_my_test_accuracy_classification = accuracy_score(y_test_classification_encoded, new_my_test_predict_classification)
new_my_test_precision_classification = precision_score(y_test_classification_encoded, new_my_test_predict_classification, average='micro')
new_my_test_recall_classification = recall_score(y_test_classification_encoded, new_my_test_predict_classification, average='micro')



print('Метрика:          F1        Accuracy        Precision          Recall')
print('Train', new_my_train_f1_classification, new_my_train_accuracy_classification, new_my_train_precision_classification, new_my_train_recall_classification)
print('Test', new_my_test_f1_classification, new_my_test_accuracy_classification, new_my_test_precision_classification, new_my_test_recall_classification)

Метрика:          F1        Accuracy        Precision          Recall
Train 0.5297504798464492 0.5297504798464492 0.5297504798464492 0.5297504798464492
Test 0.48659003831417624 0.48659003831417624 0.48659003831417624 0.48659003831417624


In [37]:

# Применение собственной реализации для регрессии
linreg_regressor = LINEARREGRESSION(lr=0.01, epochs=1000, regularization="l2", reg_strength=0.1)
linreg_regressor.fit(X_train_preprocessed.toarray(), y_train_regression)

new_my_train_predict_regression = linreg_regressor.predict(X_train_preprocessed.toarray())
new_my_test_predict_regression = linreg_regressor.predict(X_test_preprocessed.toarray())

In [38]:
# Расчет метрик
new_my_train_mse_regression = mean_squared_error(y_train_regression, new_my_train_predict_regression)
new_my_train_mae_regression = mean_absolute_error(y_train_regression, new_my_train_predict_regression)

new_my_test_mse_regression = mean_squared_error(y_test_regression, new_my_test_predict_regression)
new_my_test_mae_regression = mean_absolute_error(y_test_regression, new_my_test_predict_regression)

print('Метрика:          MSE            MAE')
print('Train', new_my_train_mse_regression, new_my_train_mae_regression)
print('Test', new_my_test_mse_regression, new_my_test_mae_regression)

Метрика:          MSE            MAE
Train 148131.00102200816 260.6823888022248
Test 231432.19908748163 271.6295261109631


##### Сравнение стандартной и собственной реализации Линейной и Логарифмической регрессий до улучшений:
В задаче классификации на обучающей выборке собственная реализация уступает стандартной:
F1-меры: 0.72 против 0.5; на тестовой выборке метрики еще хуже: 0.69 и 0.39, что говорит о плохой работе собственной реализации.
В задаче регрессии MSE и MAE собственной и стандартной реализаций одинаковые, что говорит о хорошей собственной реализации.

##### Сравнение стандартной и собственной реализации Линейной и Логарифмической регрессий после улучшения бейзлайна:
Собственная реализация для классификации также показывает не самую лучшую F1-меру на обучении (0.52 против 0.98), на тесте F1-мера также немного выше по отношению непредобработанному датасету: 0.47 против 0.79. Это говорит о том, что собственная реализация работает хуже стандартной, однако относительно непредобработанного датасета видны улучшения.
В задаче регрессии MSE и MAE у собственной реализации после улучшений хуже как на обучении, так и на тесте: на тесте MSE увеличилось до 231427 (против 81385 у стандартной реализации), а MAE до 271 (против 189).


### Вывод:
До улучшения модели справлялись с задачами, но метрики указывали на недостаточную адаптацию к данным. В классификации недообучение было явным, а в регрессии — высокая ошибка предсказаний. После улучшения классификация показала значительно лучшие результаты, но возможное переобучение на тренировочной выборке. Регрессия также существенно улучшилась, особенно на тренировочных данных. Разрыв между тренировочными и тестовыми метриками стал меньше.
В собственной реализации модели после улучшений стали показывать лучшее качество, но результаты всё ещё уступают встроенным библиотечным решениям. Это связано с упрощённой реализацией.


Также в сравнении с другими алгоритмами машинного обучения можно заметить, что Линейная регрессия и Логистическая регрессия плохо справляются с данным датасетом. 